In [1]:
import os
from sys import maxsize
import numpy as np
import matplotlib.pyplot as plt
from utils.filerw import readdat
import importlib
import utils.efcalcs as efcalcs
# import ef_percent

In [5]:
importlib.reload(efcalcs)

<module 'utils.efcalcs' from '/home/shail/work/UIC/ef-alloc-experiments/utils/efcalcs.py'>

100 per file * 10 files

In [2]:
dir_u_r10_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/"
dir_uv_r10_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/"

dir_u_r4_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/"
dir_uv_r4_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/"

dir_u_r6_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/"
dir_uv_r6_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/"

dir_uv_r6_u4 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u4/"

dir_u_r8_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r8_u2/"
dir_uv_r8_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/"



In [3]:
# failtemp = np.argwhere(ef1_map==False)[0][0]
# print(failtemp)
# print(Us[failtemp])
# print(Vs[failtemp])
# print(As[failtemp])
# print(envymap1[failtemp])

In [6]:
(en0, en1, en2) = efcalcs.ef_percent(dir_u_r10_u2)

Us shape : (1000, 2, 10)
Vs shape : (1000, 2, 5)
As shape : (1000, 2, 10)
EF0 percent :  99.9
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[  0.         156.83334623]
 [155.02036867   0.        ]]
[[  0.         156.83334623]
 [155.11827364   0.        ]]
[[  0.         156.83334623]
 [155.11827364   0.        ]]
Means: max(0, envy)
[[  0.         156.83334623]
 [155.02114089   0.        ]]
[[  0.         156.83334623]
 [155.11827364   0.        ]]
[[  0.         156.83334623]
 [155.11827364   0.        ]]
Means: min(0, envy)
[[ 0.          0.        ]
 [-0.00077223  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         68.37707329]
 [65.3157873   0.        ]]
[[ 0.         68.37707329]
 [65.15536916  0.        ]]
[[ 0.         68.37707329]
 [65.15536916  0.        ]]


In [8]:
(en0, en1, en2) = efcalcs.ef_percent(dir_u_r8_u2)

Us shape : (1000, 2, 8)
Vs shape : (1000, 2, 4)
As shape : (1000, 2, 8)
EF0 percent :  98.1
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[  0.         126.33334216]
 [122.9806306    0.        ]]
[[  0.         127.21740552]
 [123.58211117   0.        ]]
[[  0.         127.21740552]
 [123.58211117   0.        ]]
Means: max(0, envy)
[[  0.         126.48942761]
 [123.09040063   0.        ]]
[[  0.         127.21740552]
 [123.58211117   0.        ]]
[[  0.         127.21740552]
 [123.58211117   0.        ]]
Means: min(0, envy)
[[ 0.         -0.15608545]
 [-0.10977003  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.        64.1111858]
 [62.878994   0.       ]]
[[ 0.         62.71469491]
 [61.93159327  0.        ]]
[[ 0.         62.71469491]
 [61.93159327  0.        ]]


In [11]:
(en0, en1, en2) = efcalcs.ef_percent(dir_u_r6_u2)

Us shape : (1000, 2, 6)
Vs shape : (1000, 2, 3)
As shape : (1000, 2, 6)
EF0 percent :  93.5
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[ 0.         91.35345389]
 [88.96794779  0.        ]]
[[ 0.         94.27388561]
 [91.05224569  0.        ]]
[[ 0.         94.27388561]
 [91.05224569  0.        ]]
Means: max(0, envy)
[[ 0.         91.77398598]
 [89.44611016  0.        ]]
[[ 0.         94.27388561]
 [91.05224569  0.        ]]
[[ 0.         94.27388561]
 [91.05224569  0.        ]]
Means: min(0, envy)
[[ 0.         -0.42053209]
 [-0.47816237  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         55.49371266]
 [53.04071767  0.        ]]
[[ 0.         52.00652743]
 [50.2832711   0.        ]]
[[ 0.         52.00652743]
 [50.2832711   0.        ]]


In [12]:
(en0, en1, en2) = efcalcs.ef_percent(dir_u_r4_u2)

Us shape : (1000, 2, 4)
Vs shape : (1000, 2, 2)
As shape : (1000, 2, 4)
EF0 percent :  77.10000000000001
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[ 0.         56.20214644]
 [56.41082445  0.        ]]
[[ 0.         65.47889405]
 [64.33766935  0.        ]]
[[ 0.         65.47889405]
 [64.33766935  0.        ]]
Means: max(0, envy)
[[ 0.         58.45416904]
 [58.44216028  0.        ]]
[[ 0.         65.47889405]
 [64.33766935  0.        ]]
[[ 0.         65.47889405]
 [64.33766935  0.        ]]
Means: min(0, envy)
[[ 0.         -2.2520226 ]
 [-2.03133583  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         47.97520014]
 [45.98858705  0.        ]]
[[ 0.         39.40358954]
 [38.34352994  0.        ]]
[[ 0.         39.40358954]
 [38.34352994  0.        ]]


In [13]:
(en0, en1, en2) = efcalcs.ef_percent(dir_uv_r10_u2)

Us shape : (1000, 2, 10)
Vs shape : (1000, 2, 5)
As shape : (1000, 2, 10)
EF0 percent :  75.7
EF1 percent :  99.9
EF2 percent :  100.0
Means: envy
[[  0.         137.73304048]
 [134.82639453   0.        ]]
[[  0.         153.9664558 ]
 [152.27546644   0.        ]]
[[  0.         154.10125936]
 [152.27546644   0.        ]]
Means: max(0, envy)
[[  0.         142.92624834]
 [140.11198442   0.        ]]
[[  0.         153.99764655]
 [152.27546644   0.        ]]
[[  0.         154.10125936]
 [152.27546644   0.        ]]
Means: min(0, envy)
[[ 0.         -5.19320787]
 [-5.2855899   0.        ]]
[[ 0.         -0.03119075]
 [ 0.          0.        ]]
[[0. 0.]
 [0. 0.]]
Stds
[[  0.         113.72814897]
 [110.42648228   0.        ]]
[[ 0.         94.89523649]
 [90.41461064  0.        ]]
[[ 0.         94.72771526]
 [90.41461064  0.        ]]


In [14]:
(en0, en1, en2) = efcalcs.ef_percent(dir_uv_r8_u2)

Us shape : (1000, 2, 8)
Vs shape : (1000, 2, 4)
As shape : (1000, 2, 8)
EF0 percent :  77.8
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[  0.         104.18248343]
 [104.58306424   0.        ]]
[[  0.         119.98533569]
 [120.19430845   0.        ]]
[[  0.         119.98533569]
 [120.19430845   0.        ]]
Means: max(0, envy)
[[  0.         107.76165509]
 [108.48607588   0.        ]]
[[  0.         119.98533569]
 [120.19430845   0.        ]]
[[  0.         119.98533569]
 [120.19430845   0.        ]]
Means: min(0, envy)
[[ 0.         -3.57917165]
 [-3.90301165  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         88.75492692]
 [87.87904888  0.        ]]
[[ 0.         74.70661871]
 [73.26920199  0.        ]]
[[ 0.         74.70661871]
 [73.26920199  0.        ]]


In [15]:
(en0, en1, en2) = efcalcs.ef_percent(dir_uv_r6_u2)

Us shape : (1000, 2, 6)
Vs shape : (1000, 2, 3)
As shape : (1000, 2, 6)
EF0 percent :  48.699999999999996
EF1 percent :  98.0
EF2 percent :  100.0
Means: envy
[[ 0.         83.02162873]
 [74.47077309  0.        ]]
[[  0.         113.55231673]
 [107.61388393   0.        ]]
[[  0.        114.2853773]
 [109.0396012   0.       ]]
Means: max(0, envy)
[[ 0.         96.76593419]
 [90.96291689  0.        ]]
[[  0.         113.64229644]
 [107.8225121    0.        ]]
[[  0.        114.2853773]
 [109.0396012   0.       ]]
Means: min(0, envy)
[[  0.         -13.74430546]
 [-16.4921438    0.        ]]
[[ 0.         -0.08997971]
 [-0.20862817  0.        ]]
[[0. 0.]
 [0. 0.]]
Stds
[[  0.         107.64191776]
 [109.58075385   0.        ]]
[[ 0.         76.93305499]
 [76.92012276  0.        ]]
[[ 0.         76.22673622]
 [75.6337297   0.        ]]


In [16]:
(en0, en1, en2) = efcalcs.ef_percent(dir_uv_r6_u4)

Us shape : (1000, 4, 6)
Vs shape : (1000, 4, 3)
As shape : (1000, 4, 6)
EF0 percent :  0.3
EF1 percent :  95.7
EF2 percent :  100.0
Means: envy
[[ 0.         52.51730648 48.91861583 54.00261447]
 [46.0726181   0.         45.98897767 51.62481001]
 [50.31942959 52.36255125  0.         54.82918477]
 [48.17590826 44.55010919 42.30215229  0.        ]]
[[ 0.         88.19108773 86.43368698 88.07603677]
 [81.45986335  0.         84.66046425 85.56435453]
 [87.48920031 87.03391533  0.         87.43552002]
 [82.5407319  83.74663431 81.32441201  0.        ]]
[[ 0.         88.4330209  86.65376388 88.23222813]
 [81.6975953   0.         85.26887626 86.05886928]
 [87.8267959  87.11447923  0.         87.64707287]
 [82.83063933 84.15395885 81.49152848  0.        ]]
Means: max(0, envy)
[[ 0.         69.29593196 67.74162881 69.86653396]
 [64.33808956  0.         64.83052735 68.85348643]
 [68.75602495 69.33144748  0.         70.5760539 ]
 [64.64755585 64.7282923  61.82426236  0.        ]]
[[ 0.         88

In [17]:
(en0, en1, en2) = efcalcs.ef_percent(dir_uv_r4_u2)

Us shape : (1000, 2, 4)
Vs shape : (1000, 2, 2)
As shape : (1000, 2, 4)
EF0 percent :  51.5
EF1 percent :  99.9
EF2 percent :  100.0
Means: envy
[[ 0.         44.52326596]
 [41.32370711  0.        ]]
[[ 0.         75.12509966]
 [74.61377435  0.        ]]
[[ 0.         75.12509966]
 [74.67106896  0.        ]]
Means: max(0, envy)
[[ 0.         53.08134235]
 [49.65425577  0.        ]]
[[ 0.         75.12509966]
 [74.61517849  0.        ]]
[[ 0.         75.12509966]
 [74.67106896  0.        ]]
Means: min(0, envy)
[[ 0.         -8.55807639]
 [-8.33054866  0.        ]]
[[ 0.          0.        ]
 [-0.00140414  0.        ]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         62.65301931]
 [59.70746581  0.        ]]
[[ 0.         46.02593862]
 [44.43850919  0.        ]]
[[ 0.         46.02593862]
 [44.3773552   0.        ]]
